<a href="https://colab.research.google.com/github/yifan-zhou19/iso/blob/master/iso_cg_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##################
# DATA IMPORTING #
##################

from os.path import exists

!wget -O cwe119_cgd.txt https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt

print()

with open("./cwe119_cgd.txt", "r") as cgd:
  print("Source: ",cgd.readline())

--2020-04-13 19:49:31--  https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17202057 (16M) [text/plain]
Saving to: ‘cwe119_cgd.txt’

cwe119_cgd.txt      100%[===================>]  16.40M  27.1MB/s    in 0.6s    

2020-04-13 19:49:33 (27.1 MB/s) - ‘cwe119_cgd.txt’ saved [17202057/17202057]


Source:  1 CVE-2010-1444/vlc_media_player_1.1.0_CVE-2010-1444_zipstream.c cfunc 449



In [2]:
import re
re_punctuation_string = '[()\s,/.\']'
programs=[]
program = []
statement=[]
labels = []
vocabulary = [] 
remove_token=['',' ',',','NULL','\n',';']
with open("./cwe119_cgd.txt", "r") as cgd:
  lines = cgd.readlines()
  for line in lines:
    statement=[token for token in re.split(re_punctuation_string,line)if token not in remove_token]
    #print(statement)
    if statement == ['---------------------------------']:
      if program[-1] == ['0']:
        labels.append(0)
      elif program[-1] == ['1']:
        labels.append(1)
      programs.append(program[1:-2])
      for s in program[1:]:
        for t in s:
           if t not in vocabulary:
             vocabulary.append(t)
      #print(program)
      program = []
    else:
      program.append(statement)

print(len(labels))


  
print(len(vocabulary))

39753
22631


In [0]:
# shuffle data
import random
randnum = random.randint(0,100)
random.seed(randnum)
random.shuffle(programs)
random.seed(randnum)
random.shuffle(labels)

In [5]:
training_data = []
for i in range(10000):
  training_data.append((programs[i],labels[i]))
print(training_data[:2])

eval_data=[]
for i in range(10000,15000):
  eval_data.append((programs[i],labels[i]))
print(eval_data[:2])

[([['void', 'CWE122_Heap_Based_Buffer_Overflow__c_CWE806_char_ncpy_52b_goodG2BSink', 'char', '*', 'data'], ['CWE122_Heap_Based_Buffer_Overflow__c_CWE806_char_ncpy_52c_goodG2BSink', 'data'], ['void', 'CWE122_Heap_Based_Buffer_Overflow__c_CWE806_char_ncpy_52c_goodG2BSink', 'char', '*', 'data'], ['strncpy', 'dest', 'data', 'strlen', 'data'], ['printLine', 'data']], 0), ([['recvResult', '=', 'recv', 'connectSocket', 'inputBuffer', 'CHAR_ARRAY_SIZE', '-', '1', '0'], ['if', 'recvResult', '==', 'SOCKET_ERROR', '||', 'recvResult', '==', '0'], ['inputBuffer[recvResult]', '=', '\\0'], ['data', '=', 'atoi', 'inputBuffer'], ['if', 'data', '>=', '0']], 1)]
[([['void', 'badSource', 'wchar_t', '*', '&data'], ['wchar_t', '*', 'dataBuffer', '=', 'wchar_t', '*', 'ALLOCA', '100*sizeof', 'wchar_t'], ['data', '=', 'dataBuffer;'], ['badSource', 'data'], ['wchar_t', 'dest[50]', '=', 'L"";']], 1), ([['data', '=', 'NULL;'], ['data[0]', '=', 'L', '\\0'], ['wcscpy', 'data', 'source'], ['printWLine', 'data']], 0)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#look up table  
#using one-hot embedding
vocabulary_size = len(vocabulary)
def look_up_table(word_idx):
  x = torch.zeros(vocabulary_size).long()
  x[word_idx] = 1.0
  return x
word2idx = {w: idx +1 for (idx, w) in enumerate(vocabulary)}
idx2word = {idx+1: w for (idx, w) in enumerate(vocabulary)}
print(look_up_table(word2idx['1']))

tensor([0, 0, 0,  ..., 0, 0, 0])


In [7]:
context_size = 20
def get_programcontext(program):
  id = 0
  contexts =torch.zeros(context_size,1).long()
  for p in program:
    for t in p:
      if id < context_size:
        contexts[id]= word2idx[t]
        id = id +1
      else:
        return contexts
  return contexts
get_programcontext(programs[4920])

tensor([[4078],
        [   5],
        [ 743],
        [4079],
        [4080],
        [   5],
        [ 168],
        [4078],
        [  99],
        [ 168],
        [4081],
        [  99],
        [ 539],
        [4082],
        [   5],
        [ 127],
        [ 638],
        [4080],
        [ 352],
        [ 151]])

In [33]:


torch.manual_seed(1)
embedding_dim =15
class ProgrammingML(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(ProgrammingML, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.conv1 = nn.Conv1d(1, 9, (5,embedding_dim),padding =(2,0))
    self.conv2 = nn.Conv2d(9, 5, (2,1))
    self.fc1 = nn.Linear(5*19, 2)

  def forward(self, inputs):
    #print(inputs.size())
    embeds = self.embeddings(inputs.view(-1))
    #print(embeds.size())
    out = self.conv1(embeds.view(1,1,context_size,embedding_dim))
    #print(out.size())
    out = self.conv2(out)
    #print(out.size())
    out = F.softmax(self.fc1(out.view(-1)),dim=0)
    return out


losses = []
loss_function = nn.BCELoss()
model = ProgrammingML(vocabulary_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
  losses = 0
  for program, label in training_data:
   # print(program)
   # print(label)
   if label ==1:
     out_labels = torch.FloatTensor([0,1])
   else:
     out_labels  = torch.FloatTensor([1,0])

   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(out.size())
   #print(f'Out {out}')
   #print(out_labels)
   loss = loss_function(out, out_labels )
   #print(f'Loss {loss.item()}')
   loss.backward()
   optimizer.step()
   #print(f'Loss {loss.item()}')
   losses +=loss.item()
  print(f'Loss at epo {epoch}: {losses/len(training_data)}')

Loss at epo 0: 0.44414275987961155
Loss at epo 1: 0.32658981557295114
Loss at epo 2: 0.28461441633015255
Loss at epo 3: 0.2626896175176947
Loss at epo 4: 0.2482062248843443
Loss at epo 5: 0.23725426327071156
Loss at epo 6: 0.22817165221168015
Loss at epo 7: 0.22065010371772859
Loss at epo 8: 0.21416260230483358
Loss at epo 9: 0.2081609038558893
Loss at epo 10: 0.2025424870689728
Loss at epo 11: 0.19763052361450603
Loss at epo 12: 0.1933149495722091
Loss at epo 13: 0.189573594894691
Loss at epo 14: 0.18661215077272103
Loss at epo 15: 0.18396923899878423
Loss at epo 16: 0.181193152381992
Loss at epo 17: 0.1786337698992101
Loss at epo 18: 0.17636177691284735
Loss at epo 19: 0.17371436395053277


In [30]:
with torch.no_grad():
  accurate = 0
  for program, label in eval_data:
   # print(program)
   # print(label)
   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(f'{out}:{label}')
   if(out[0]>0.5):
     if(label == 0):
       accurate =accurate+1
   elif(out[1]>0.5):
     if(label == 1):
       accurate = accurate+1
  print(f'Accuracy: {accurate/len(eval_data)}')   

Accuracy: 0.8628


In [34]:
with torch.no_grad():
  accurate = 0
  for program, label in eval_data:
   # print(program)
   # print(label)
   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(f'{out}:{label}')
   if(out[0]>0.5):
     if(label == 0):
       accurate =accurate+1
   elif(out[1]>0.5):
     if(label == 1):
       accurate = accurate+1
  print(f'Accuracy: {accurate/len(eval_data)}')  

Accuracy: 0.8676
